In [1]:
import pandas as pd

import tables

times = [pd.to_timedelta(5, 'm'), pd.to_timedelta(3, 'h')]
df = pd.read_csv('../data/first_detection_times.csv')
df['time'] = pd.to_timedelta(df['time'])
df.head()

,campaign_id,fuzzer,subject,defect,time
0,10,Zest,Bcel,B0,0 days 00:42:48.816000
1,100,Zeugma-1PT,Rhino,R0,0 days 01:34:51.614000
2,100,Zeugma-1PT,Rhino,R1,0 days 00:44:59.002000
3,100,Zeugma-1PT,Rhino,R2,0 days 00:49:16.492000
4,100,Zeugma-1PT,Rhino,R3,0 days 00:00:13.996000


In [2]:
def create_detection_count_table(df, times):
    elements = []
    for time in times:
        counts = df.groupby(['campaign_id', 'fuzzer', 'subject'])['time'] \
            .apply(lambda x: (x < time).sum()) \
            .reset_index(name='detected')
        counts['time'] = time
        elements.append(counts)
    return pd.concat(elements)


counts = create_detection_count_table(df, times)
display(counts.head())

,campaign_id,fuzzer,subject,detected,time
0,8,BeDivFuzz-Simple,Bcel,0,0 days 00:05:00
1,9,BeDivFuzz-Structure,Bcel,0,0 days 00:05:00
2,10,Zest,Bcel,0,0 days 00:05:00
3,11,Zeugma-Linked,Bcel,1,0 days 00:05:00
4,12,Zeugma,Bcel,0,0 days 00:05:00


In [3]:
import report_util


def select(data, time, subject):
    data = data[data['time'] == time]
    return data[data['subject'] == subject]


def create_rows(data, value, target_fuzzer='Zeugma-Linked'):
    fuzzers = sorted(data['fuzzer'].unique())
    rows = []
    values1 = data[data['fuzzer'] == target_fuzzer][value]
    significance_level = 0.05 / (len(fuzzers) * (len(fuzzers) - 1) / 2)
    for fuzzer in fuzzers:
        values2 = data[data['fuzzer'] == fuzzer][value]
        sig = 'color: red;' if report_util.mann_whitney(values1, values2) < significance_level else ''
        rows.append([fuzzer, values2.median(), sig])
    return pd.DataFrame(rows, columns=['fuzzer', 'value', 'sig'])


def create_sig_table(data, value):
    frames = []
    for time, subject in data[['time', 'subject']].drop_duplicates().itertuples(index=False, name=None):
        selected = select(data, time, subject)
        frame = create_rows(selected, value)
        frame['time'] = time
        frame['subject'] = subject
        frames.append(frame)
    return pd.concat(frames)


table = create_sig_table(counts, 'detected')
medians = tables.pivot(table, 'subject', 'fuzzer', 'value')
sigs = tables.pivot(table, 'subject', 'fuzzer', 'sig')
styler = tables.style_table(medians, precision=1, axis=0).apply(lambda _: sigs, axis=None)
display(styler)

In [4]:
print(styler.to_latex(multicol_align='c', hrules=True, multirow_align='t', convert_css=True))

\begin{tabular}{lrrrrrrrr}
\toprule
Subject & \multicolumn{2}{c}{Bcel} & \multicolumn{2}{c}{Closure} & \multicolumn{2}{c}{Nashorn} & \multicolumn{2}{c}{Rhino} \\
 & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H \\
Fuzzer &  &  &  &  &  &  &  &  \\
\midrule
BeDivFuzz-Simple & {\cellcolor{yellow}} 0.0 & {\cellcolor{yellow}} 1.0 & 0.0 & \color{red} 1.0 & {\cellcolor{yellow}} 0.0 & \color{red} 0.0 & {\cellcolor{yellow}} 1.0 & \color{red} 1.0 \\
BeDivFuzz-Structure & {\cellcolor{yellow}} 0.0 & {\cellcolor{yellow}} 1.0 & \color{red} 0.0 & \color{red} 1.0 & {\cellcolor{yellow}} 0.0 & \color{red} 0.0 & {\cellcolor{yellow}} 1.0 & \color{red} 2.0 \\
RLCheck & --- & --- & 0.0 & \color{red} 1.0 & {\cellcolor{yellow}} 0.0 & \color{red} 0.0 & {\cellcolor{yellow}} 1.0 & \color{red} 1.0 \\
Zest & {\cellcolor{yellow}} 0.0 & {\cellcolor{yellow}} 1.0 & \color{red} 0.0 & 1.0 & {\cellcolor{yellow}} 0.0 & \color{red} 0.0 & {\cellcolor{yellow}} 1.0 & \color{red} 2.0 \\
Zeugma & {\cellcolor{yellow}} 0.0 & {\cellcolor